In [1]:
import pandas as pd

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

In [2]:
import os
from functools import reduce, partial
from utils import read_yaml
from pathlib import Path
import usuarios
import backup
import vpn_connection as vpn
import usuarios
import acidentes
import acidentes_filtrar

root_dir = Path().resolve().parent

# Importa configurações do sistema
cfg = read_yaml(root_dir / 'config/config.yaml')
secrets = read_yaml(root_dir / 'config/secrets.yaml')
codigos_desativados = read_yaml(root_dir / 'config/codigos_desativados_conversao.yaml')
fatores_params = read_yaml(root_dir / 'config/fatores_risco_classificacao.yaml')
fatores_params_reshaped = acidentes.reshape_fatores_params(fatores_params)

# Importa dados que populam as opções do formulário de inscrição
opcoes = usuarios.import_google_spreadsheet(cfg['FORM_INSC_OPCOES']['ID_GSHEET'])

# Importa listas de inscrições e cancelamentos e compila lista de usuários ativos
inscricoes = usuarios.import_google_spreadsheet(cfg['FORM_INSC']['ID_GSHEET'])
cancelamentos = usuarios.import_google_spreadsheet(cfg['FORM_CANCEL']['ID_GSHEET'])
inscricoes_compilado = usuarios.compila_inscricoes(df_inscricao=inscricoes, df_cancelamento=cancelamentos)
inscricoes_compilado_novos_codigos = usuarios.update_codigos_desativados(inscricoes_compilado, codigos_desativados)

# Realiza backup dos dados provenientes dos formulários de inscrição e cancelamento
backup_dir = str(root_dir / 'data/backup')

backup.backup_csv_new_file(opcoes, directory=backup_dir, filename='opcoes_form_insc.csv')
backup.backup_csv(inscricoes, backup_path=os.path.join(backup_dir, 'inscricoes.csv'))
backup.backup_csv(cancelamentos, backup_path=os.path.join(backup_dir, 'cancelamentos.csv'))
backup.backup_csv(inscricoes_compilado_novos_codigos, backup_path=os.path.join(backup_dir, 'inscricoes_compilado.csv'))

Reading configuration file: C:\Users\joao.reis\Pywinauto recorder\config.ini
Window filtering mode: ignore_windows
Ignored windows: []


In [3]:
# Tenta conectar à VPN
# vpn.try_connection_forticlient_vpn(vpn_path=cfg['VNP_PATH'],
#                                     user=secrets['VNP_USER'],
#                                     password=secrets['VNP_PASSWORD'],
#                                     url_test_connection=cfg['VPN_URL_TEST_CONNECTION'])

# Importa novas CATs
cats = acidentes.cat_extrair()

In [4]:
# Trata CATs
cat_atribui_fatores_risco_partial = partial(acidentes.cat_atribui_fatores_risco,
                                            fatores_params_reshaped=fatores_params_reshaped)

functions_list = [acidentes.cat_eliminar_ja_alertadas,
                    acidentes.cat_converter_datas,
                    acidentes.cat_formatar_horas,
                    acidentes.cat_formatar_strings,
                    acidentes.cat_cid_uppercase,
                    acidentes.cat_novas_colunas,
                    acidentes.cat_uorg_local_acidente,
                    acidentes.cat_secao_cnae_local_acidente,
                    acidentes.cat_formatar_datas,
                    acidentes.cat_identifica_recibo_raiz,
                    acidentes.cat_mantem_recibo_ultima_reabertura,
                    cat_atribui_fatores_risco_partial,
                    acidentes.cat_compila_fatores_risco,
                    acidentes.cat_atribui_consequencia,
                    acidentes.cat_inserir_descricoes
                    ]

cats_tratadas = reduce(lambda x, y: y(x), functions_list, cats)

In [5]:
destinatario = inscricoes_compilado_novos_codigos.iloc[1]
destinatario

Timestamp_incricao                                                   2022-08-23 13:51:56
E-mail                                                    joao.havelange@economia.gov.br
Consequência do acidente                                                           Óbito
Abrangência geográfica                                    Unidade da Federação (ex.: MG)
UF                                                                                    DF
UORG                                                                                 NaN
Tipo de acidente            Acidentes típicos, Doenças do Trabalho, Acidentes de Trajeto
Setores econômicos                                                                   Não
Seção CNAE                                                                           NaN
Fator de risco                                                                       Não
Fatores de risco                                                                     NaN
Timestamp_cancelament

In [9]:
destinatario = inscricoes_compilado_novos_codigos.iloc[1]

funcoes_filtra_cats = [acidentes_filtrar.uf,
                       acidentes_filtrar.uorg,
                       acidentes_filtrar.tpacid,
                       acidentes_filtrar.consequencias,
                       acidentes_filtrar.risco,
                       acidentes_filtrar.cnae,]

funcoes_filtra_cats_partial = [partial(function, usuario=destinatario) for function in funcoes_filtra_cats]

cats_filtradas = reduce(lambda x, y: y(x), funcoes_filtra_cats_partial, cats_tratadas)
cats_filtradas

,tpinsc,nrinsc,localtabgeral_tpinsc,localtabgeral_nrinsc,inporte,cnae_localtabgeral,razao_social,municipio_empregador,sguf_empregador,tpinsc_estab_local_acidente,nrinsc_estab_local_acidente,razao_social_estab_local_acidente,cnae_local_acidente,municipio_estab_local_acidente,sguf_estab_local_acidente,cpftrab,nistrab,matricula,nmtrab,dtadm,codcbo,nmcargo,sexo,grauinstr,racacor,dtnascto,codcateg,vrsalfx,tpacid,tpcat,dtacid,hracid,hrstrabantesacid,tplocal_acidente,dslocal_acidente,tplograd_local_acidente,dslograd_local_acidente,nr_lograd_local_acidente,complemento_local_acidente,bairro_local_acidente,cep_local_acidente,municipio_local_acidente,sguf_local_acidente,pais_local_acidente,codpostal_local_acidente,codagntcausador,codsitgeradora,codparteating,lateralidade,dsclesao,dsccomplesao,diagprovavel,codcid,obsatestado,indinternacao,indcatobito,dtobito,indafast,durtrat,dtatendimento,hratendimento,nmemit,ideoc,nroc,ufoc,iniciatcat,indretif,nrrecibo,procemi,meta_row_key,meta_nr_recibo,indcomunpolicia,nrRecCatOrig,idade_DTAcidente,DTEmissaoCAT,CDEmitenteCAT,NRCAT,codcidCategoria,ultdiatrab,uorg_local_acidente,uf_uorg_local_acidente,secao_cnae_local_acidente,recibo_raiz,codsitgeradora_fr,codagntcausador_fr,dsclesao_fr,codcid_fr,codcidCategoria_fr,CDFatorAmbiental,Consequencia,ds_tpacid,ds_tplocal_acidente,ds_tplograd_local_acidente,ds_municipio_local_acidente,ds_pais_local_acidente,ds_codagntcausador,ds_codsitgeradora,ds_codparteating,ds_lateralidade,ds_dsclesao,ds_codcid,ds_ideoc,ds_codcbo,ds_grauinstr,ds_racacor,ds_codcateg,ds_tpinsc,ds_localtabgeral_tpinsc,ds_cnae_localtabgeral,ds_municipio_empregador,ds_tpinsc_estab_local_acidente,ds_cnae_local_acidente,ds_municipio_estab_local_acidente,ds_CDEmitenteCAT,ds_iniciatcat,ds_tpcat,ds_indretif,ds_procemi,ds_inporte
337,1,16010431,1.0,16010431011707,NaN,4771701,SANTA MARTA DISTRIBUIDORA DE DROGAS LTDA,5300108,DF,1.0,16010431011707,SANTA MARTA DISTRIBUIDORA DE DROGAS LTDA,4771701,5300108,DF,03428924100,16615973244,SC01S0010751,WESLEY AIRES DA FONSECA,09/12/2016,141415,GESTOR AUXILIAR DE LOJA,M,7,3,13/03/1991,101.0,1435.73,1,1,19/08/2022,14:40,00:00,4,VIA PUBLICA,None,QSA 5,74,Df Km Eptc Pisto Sul Prximo,Taguatinga Sul,70297400,5300108,DF,None,None,200004600,200004600,759000000,0,706050000,None,None,R571,None,N,S,19/08/2022,N,0000,19/08/2022,14:41,ANA VERUSCA TORRES,1,15463,DF,1,1,None,1,202208220000000016691184985,1.1.0000000015842608801,S,None,31.0,22/08/2022,1,1.1.0000000015842608801,R57,NaN,014000000,DF,G,1.1.0000000015842608801,NaN,NaN,NaN,NaN,NaN,[],[Óbito],Típico,Via pública,NaN,Brasília,NaN,Impacto de pessoa contra objeto em movimento,Impacto de pessoa contra objeto em movimento,"Localização da lesão, NIC",Não aplicável,Lesões múltiplas,Choque hipovolêmico,Conselho Regional de Medicina - CRM,Gerente de loja e supermercado,Ensino médio completo,Parda,"Empregado - Geral, inclusive o empregado público da administração direta ou indireta contratado pela CLT",CNPJ,CNPJ,"Comércio varejista de produtos farmacêuticos, sem manipulação de fórmulas",Brasília,CNPJ,"Comércio varejista de produtos farmacêuticos, sem manipulação de fórmulas",Brasília,Empregador,Empregador,Inicial,Original,Aplicativo do empregador,NaN
